In [1]:
from neuron import h, units, rxd, load_mechanisms
from neuron.units import mV, ms, um, mM
import matplotlib
from matplotlib import pyplot as plt
h.load_file("stdrun.hoc")
%matplotlib inline
from itertools import product
import plotly.express as px
import pandas as pd
import os

--No graphics will be displayed.


In [2]:
# Turns off scientific notation and shift in plots
matplotlib.rc('axes.formatter', useoffset=False)

In [2]:
! nrnivmodl
load_mechanisms("C:/Users/rober/OneDrive - University of Florida/Documents/Research/Pancreatic_islet_model/Model-of-Pancreatic-Islets/Test")

x86_64-w64-mingw32-gcc.exe -DDLL_EXPORT -DPIC -I/cygdrive/c/nrn/bin//../include -I/cygdrive/c/nrn/bin//../src/scopmath -I/cygdrive/c/nrn/bin//../src/nrnoc -I/cygdrive/c/nrn/bin//../src/oc  -c mod_func.c
nocmodl one
x86_64-w64-mingw32-gcc.exe -DDLL_EXPORT -DPIC -I/cygdrive/c/nrn/bin//../include -I/cygdrive/c/nrn/bin//../src/scopmath -I/cygdrive/c/nrn/bin//../src/nrnoc -I/cygdrive/c/nrn/bin//../src/oc  -c one.c
nocmodl steady_k
x86_64-w64-mingw32-gcc.exe -DDLL_EXPORT -DPIC -I/cygdrive/c/nrn/bin//../include -I/cygdrive/c/nrn/bin//../src/scopmath -I/cygdrive/c/nrn/bin//../src/nrnoc -I/cygdrive/c/nrn/bin//../src/oc  -c steady_k.c
x86_64-w64-mingw32-gcc.exe  -shared mod_func.o one.o steady_k.o \
  -L/cygdrive/c/nrn/bin//../bin -lnrniv -lpthread -o 664.nrnmech.dll
#rebase -b 0x64000000 -v nrnmech.dll

nrnmech.dll was built successfully.


Translating one.mod into one.c
Thread Safe
Translating steady_k.mod into steady_k.c
Thread Safe


True

In [3]:
# This function assumes the ECS is a cube and each of the voxels are cubes
# It calculates how many voxels span across on of the sides of the cube
# The possible indicies for the voxels will the set containing all possible
# combinations of the numbers from 0 to (length in voxels - 1)
# This comes out as a iterable object of tuples, so we convert it and each
# of its elements to lists.

def create_index_list(species, ecs_name):
    length_in_voxels = len(species[ecs_name].states3d[0][0])
    coord_list = list(product(range(length_in_voxels), repeat = 3))
    coord_list = [list(coord) for coord in coord_list]
    return coord_list

def create_rec_conc_vects(time_vect_name, coord_list, species, ecs_name):
    vect_dict = {time_vect_name : h.Vector().record(h._ref_t)}
    num_voxels = len(coord_list)
    for i in range(num_voxels):
        vect_dict[str(species) + str(coord_list[i])] = h.Vector()
        vect_dict[str(species) + str(coord_list[i])].record(species[ecs_name].node_by_ijk(coord_list[i][0], coord_list[i][1], coord_list[i][2])._ref_value)
    return vect_dict

def voxel_conc_plots(time_vect_name, coord_list, vector_dictionary, species, image_directory):
    os.mkdir("Plots/" + image_directory)
    num_voxels = len(coord_list)
    for i in range(num_voxels):
        plt.plot(vector_dictionary[time_vect_name], vector_dictionary[str(species) + str(coord_list[i])])
        plt.title(str(species) + str(coord_list[i]))
        plt.savefig("Plots/" + image_directory + "/" + str(species) + str(coord_list[i]) + ".png")
        plt.close()

def run_and_visualize_sim(species, ecs_name, time_vect_name, simulation_time, image_directory):
    coord_list = create_index_list(species, ecs_name)
    vector_dictionary = create_rec_conc_vects(time_vect_name, coord_list, species, ecs_name)
    h.finitialize()
    h.continuerun(simulation_time * ms)
    voxel_conc_plots(time_vect_name, coord_list, vector_dictionary, species, image_directory)
    print(min(vector_dictionary[str(species) + "[1, 1, 1]"]), max(vector_dictionary[str(species) + "[1, 1, 1]"])) 

# Testing multiple extracellular spaces

Throughout the first interation of this notebook I learned that if you have more than one extracellular space that things initialize improperly. This can be replicated by creating one extracellular space and running a simulation and then when you create the next and add a species, the initial concentrations in the `states3d` matrix will be zeros regardless of the initial value you input but you will get the right initial value when checking `species.initial` as `initial` is a property of the species and `states3d` is an object of the extracellular space. I thought this was because at first the coordinates of my extracellular spaces were identical meaning if they exist is some more abstract space in the background of NEURON, that they are overlapping. So I tried to create a second space with different coordinates and this was still the case. When instead of creating a second space I created a new species in the same space (Ins_2_test_const_rate_same_ecs) and the results were as they should be despite the initialization being different. The issue might have something to do with files that are created when a reaction-diffusion simulation is ran (name e.g., "rxddllb77c608a-f973-11eb-8286-c858c0244ed2.so"). Two were created when I ran a simulation with a singular extracellular space. I ran another identical simulation and two more were created and the results were the same (Insulin_test_const_rate and Insulin_test_const_rate_2). 

# Defining reactions with range variables from mod files

I think the weird issue we were having with using the JIS variable from the one mechanism might be because of it's small value causing the integrator to have issues. 

## Test case with k_steady mod file

To test this I will first run a simulation using the `ik` range variable in the steady_k mechanism. This worked as expected, I used a initial concentration of 100 and it increased to 2100 over the 2000 ms. I added a line of code that outputs the max and min concentrations during the simulations in the central voxel where the section is.

## Test case using comparable value to JIS

Sometimes the integrator will run into issues when using very small values. JIS initially is 9.608596975100945e-09. NEURON recommends using variable time step integration to deal with this but we haven't been able to successfully use it with our one mechanism. The steady_k will be a good test case since there is no derivative block. Therefore if we use the variable time step here it will effect solely the rxd rate equation and not interact with the mod file. First I will try running a simulation with a small value without variable-step enabled and then I will try it with if I see errors. 

### Run 1

No errors were seen with a small initialized value (ik_test_small_init). I am going to see if there is any effect of the argument `atolscale` by removing it (test_const_rate_no_atolscale). The first thing I noticed is the initial values in `states3d` are not zero as they were before, but rather 1e-9 which is what I set it to. After running the simulation, all appears as it should.

## Re-testing simulation using one mechanism and JIS

The plots of this simulation (test_JIS_no_atol) are odd. They show an immediate drop from around 95 to 87, but the values shown for the min and max are 48.04298487550472 48.04298494148047 respectively. The values shown in the plot are to the negative 8th power and appear to match the data. The scientific notation on the border of the plots appears to be a shift. In the figure it says 1e-8+4.8042984e1 which means the values shown are shifted by 48.042984. In the simulation with the one mechanism and no rxd, we saw a constant decrease to below 48 in a 2000 simulation. I will try to reintroduce the `atolscale` argument and then try variable timestep.

### `atolscale`

This had no effect (test_JIS_atol_set_var_disabled)

### Setting absolute and relative tolerances in CVode settings

I changed the integration method back to cnexp and will try to use variable time step integration with that since using it with non-variable time step didn't do anything (test_JIS_atol_cnexp_var_time_dis). These effprts made no changes to the end results (test_JIS_atol_cnexp_var_time, test_JIS_atol_derivimplicit_var_time, test_JIS_atol_derivimplicit). The only difference seems to be the runtime. The variable time-step simulations finish running in 7 - 8 seconds whereas the fixed-timestep simulations take 20 - 25 seconds.


In [9]:
48.04298494148047 - 48.04298487550472

6.597574753186564e-08

In [4]:
my_cell = h.Section(name = "my_cell")
my_cell.insert("one")

# Give cell length and diameter of 10 micrometers
my_cell.pt3dadd(10,15,15,10)
my_cell.pt3dadd(20,15,15,10)

# Create extracellular space that is a 30x30x30 cube made up of 10x10x10 voxels 
ecs = rxd.Extracellular(0,0,0,30,30,30, dx = (10, 10, 10))

# Leave d = 0 so no diffusion meaning no insulin should flow from one voxel to another
# The concentration curves of all voxels should be the same
test = rxd.Species(ecs, name = "test", d = 0, initial = 48.04298494148047, atolscale=1e-12)

In [4]:
test.initial

48.04298494148047

In [5]:
test[ecs].states3d

array([[[48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494]],

       [[48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494]],

       [[48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494],
        [48.04298494, 48.04298494, 48.04298494]]])

In [6]:
# (my_cell(0.5).one.JIS/vc)-fb*Ins
vc = 1e-13
fb = 2000
my_cell(0.5).one.JIS = 9.608596975100945e-09
print(my_cell(0.5).one.JIS)
test_rate_rxn_JIS = rxd.Rate(test, (my_cell(0.5).one.JIS/vc)-fb*test, regions = ecs)

9.608596975100945e-09


In [7]:
h.CVode().active(False)
# h.CVode().rtol(1e-12)
# h.CVode().atol(1e-12)
run_and_visualize_sim(test, ecs, "t_vect", 2000, "test_JIS_atol_derivimplicit")

48.04298487550472 48.04298494148047


This works as expected if you look at the figures in "Plots/Insulin_test_const_rate". The value we need from the `one` mechanism is `JIS`. JIS has a very small value of JIS=9.608596975100945e-09 which is pretty difficult to keep track of so I will create a new cell and insert the `steady_k` mechanism which is a simple mod file:

'''NEURON {
	SUFFIX steady_k
	USEION k WRITE ik VALENCE 1
	RANGE ik, rate
}

PARAMETER {
    rate    (mA/cm2)
}

INITIAL {
    rate = 1
}
 
ASSIGNED {
    ik     (mA/cm2)
}
 
BREAKPOINT {
    ik = rate
}'''

I will make the concentration change equal to `ik` so we should get the same output as before since `ik` is equal to 1. Also, I will make some of the above code into functions to help things move faster

In [11]:
my_cell_2 = h.Section(name = "my_cell_2")
my_cell_2.insert("steady_k")

my_cell_2

In [12]:
my_cell_2(0.5).steady_k.ik

0.0

The value appears to be zero so I will try to set it myself to 1. Also to be safe I will create a new extracellular space as well as species and reaction.

In [13]:
my_cell_2(0.5).steady_k.ik = 1
my_cell_2(0.5).steady_k.ik

1.0

Location of section and all other parameters will be kept the same as above

In [14]:
# Give cell length and diameter of 10 micrometers
my_cell_2.pt3dadd(10,15,15,10)
my_cell_2.pt3dadd(20,15,15,10)
ecs_2 = rxd.Extracellular(0,0,0,30,30,30, dx = (10, 10, 10))
# Leave d = 0 so no diffusion meaning insulin should stay right outside of membrane
ik_test = rxd.Species(ecs_2, name = "ik_test", d = 0, initial = 1e-9, atolscale=1e-10)

In [15]:
ik_test.initial

1e-09

In [16]:
ik_test[ecs_2].states3d

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [17]:
ik_test_rate = rxd.Rate(ik_test, my_cell_2(0.5).steady_k.ik, regions=ecs_2)

In [18]:
run_and_visualize_sim(ik_test, ecs_2, "t_vect", 2000, "ik_test_small_init")

1e-09 2000.0000000038933


The initial concentrations aren't being set properly in the states3d matrix. Maybe this has to do with them being located in the same space. Lets try to delete the `ecs`, species, section, etc and try to create an equal sized ecs in a different location. 

As I wrote the code above `ecs_2` did not pop up as a variable as I was typing. There are some temporary files created when I create extracellular spaces and species (e.g., `rxddll7a39935e-f95d-11eb-9f65-c858c0244ed2.so`). I will try to delete all of them and run this again to see if maybe the first extracellular space is saved into one of these files and called whenever I create an instance of extracellular space.

Note: It won't let me delete the files because they are in use. Either from the terminal, the notebook, or file explorer.I am going to close out and use on extracellular space and delete the sections, species, etc in between

In [ ]:
# Redefine the rate reaction
I_secretion_const = rxd.Rate(Ins, my_cell(0.5).one.JIS, regions = ecs)